# Run a grid of models for comparison to cloudy

This version runs things to convergence

This version does not contain the summary tables

In [1]:
import os
os.getcwd()

'/Users/long/Projects/Python/release-models/cloudy/python'

In [2]:
import subprocess
import numpy as np
import shutil
from glob import glob

In [3]:
base='''System_type(star,cv,bh,agn,previous)                   bh

### Parameters for the Central Object
Central_object.mass(msol)                  0.8
Central_object.radius(cm)                  1e10
Binary.mass_sec(msol)                           0.4
Binary.period(hr)                             1000

### Parameters for the Disk (if there is one)
Disk.type(none,flat,vertically.extended,rmin>central.obj.rad)                 none

### Parameters for Boundary Layer or the compact object in an X-ray Binary or AGN
Central_object.radiation(yes,no)                  yes
Central_object.rad_type_to_make_wind(bb,models,power,cloudy,brems,mono)               cloudy
Central_object.luminosity(ergs/s)          %.3e
Central_object.power_law_index             -0.9
Central_object.geometry_for_source(sphere,lamp_post,bubble)               sphere
Central_object.cloudy.low_energy_break(ev)                .136
Central_object.cloudy.high_energy_break(ev)               20000

### Parameters describing the various winds or coronae in the system
Wind.number_of_components                  1
Wind.type(SV,star,hydro,corona,kwd,homologous,shell,imported)                shell

### Parameters associated with photon number, cycles,ionization and radiative transfer options
Photons_per_cycle                          10000000
Ionization_cycles                          30
Spectrum_cycles                            0
Wind.ionization(on.the.spot,ML93,LTE_tr,LTE_te,fixed,matrix_bb,matrix_pow,matrix_est)           matrix_pow
Line_transfer(pure_abs,pure_scat,sing_scat,escape_prob,thermal_trapping,macro_atoms_escape_prob,macro_atoms_thermal_trapping)     thermal_trapping
Wind.radiation(yes,no)                           no
Atomic_data                                data/standard80.dat
Surface.reflection.or.absorption(reflect,absorb,thermalized.rerad)              reflect
Wind_heating.extra_processes(none,adiabatic,nonthermal,both)                 none

### Parameters for Domain 0
Shell.wind_mdot(msol/yr)                   4.7e-20
Shell.wind.radmin(cm)                      1e11
Shell.wind.radmax(cm)                      1.00000000001e11
Shell.wind_v_at_rmin(cm)                   1
Shell.wind.v_at_rmax(cm)                   1.000010
Shell.wind.acceleration_exponent           1
Wind.t.init                                10000
Wind.filling_factor(1=smooth,<1=clumped)   1

### Parameters for Reverberation Modeling (if needed)
Reverb.type(none,photon,wind,matom)                 none

### Other parameters
Photon_sampling.approach(T_star,cv,yso,AGN,tde_bb,min_max_freq,user_bands,cloudy_test,wide,logarithmic)          cloudy_test
Photon_sampling.low_energy_limit(eV)            0.13
Photon_sampling.high_energy_limit(eV)                 1e8
'''

In [4]:
def run_one(log_ip=-8,version='87h'):

    if log_ip<0:
        pfname='xx_m%03.2f' % (-log_ip)
    else:
        pfname='xx_p%03.2f' % (log_ip)
    

    f=open(pfname+'.pf','w')
    
    xlum=1e21 * 10**(log_ip+8.0)

    print(xlum)
    
    f.write(base % xlum)
    f.close()

    py='py%s' % version
  

    xcommand='mpirun -np 8 %s -p %s > %s.out.txt' % (py,pfname,pfname)
    print(xcommand)
    subprocess.run(xcommand,shell=True, check=True)

    try:
        signame= '%s.sig' % pfname
        f=open(signame)
    except:
        print('Error: Could not open %s ' % signame)
        return 1
        

    lines=f.readlines()
    last_line=lines[-1]
    print(last_line)
    if last_line.count('COMPLETE'):
        windsave2table = 'windsave2table%s' % version
        xcommand='%s %s >> %s.out.txt' % (windsave2table,pfname,pfname)
        # print(xcommand)
        subprocess.run(xcommand,shell=True, check=True)
        return 0
    return 1
              
run_one()    

1e+21
mpirun -np 8 py87h -p xx_m8.00 > xx_m8.00.out.txt
Fri Mar  8 17:39:31 2024    100.4 COMPLETE             xx_m8.00



0

In [5]:
def do_many(log_ipmin=-5,log_ipmax=8,npts=10,version='87h'):
    log_ip=np.linspace(log_ipmin,log_ipmax,npts)
    # print(log_ip)
    i=1
    for one in log_ip:
        if one<0:
            pfname='xx_m%03.2f' % (-one)
        else:
            pfname='xx_p%03.2f' % (one)
        ival=run_one(one,version)
        if ival==0:
            os.remove(pfname+'.wind_save')
            shutil.rmtree('diag_%s' % pfname)
            files = glob('%s*spec*' % pfname)
            # print(files)
            for one_file in files:
                os.remove(one_file)
            print('Finished %s successfully (%d/%d)' % (pfname,i,len(log_ip)))
        else:
            print('Failed on %s  (%d/%d)' % (pfname,i,len(log_ip)))
        i+=1
                  
        
            
            
            

do_many(npts=100,version='87h')
    
    
    

1e+24
mpirun -np 8 py87h -p xx_m5.00 > xx_m5.00.out.txt
Fri Mar  8 17:41:23 2024    110.6 COMPLETE             xx_m5.00

Finished xx_m5.00 successfully (1/100)
1.3530477745798076e+24
mpirun -np 8 py87h -p xx_m4.87 > xx_m4.87.out.txt
Fri Mar  8 17:43:14 2024    110.1 COMPLETE             xx_m4.87

Finished xx_m4.87 successfully (2/100)
1.8307382802953697e+24
mpirun -np 8 py87h -p xx_m4.74 > xx_m4.74.out.txt
Fri Mar  8 17:45:04 2024    109.2 COMPLETE             xx_m4.74

Finished xx_m4.74 successfully (3/100)
2.477076355991709e+24
mpirun -np 8 py87h -p xx_m4.61 > xx_m4.61.out.txt
Fri Mar  8 17:46:54 2024    109.2 COMPLETE             xx_m4.61

Finished xx_m4.61 successfully (4/100)
3.3516026509388406e+24
mpirun -np 8 py87h -p xx_m4.47 > xx_m4.47.out.txt
Fri Mar  8 17:48:41 2024    106.1 COMPLETE             xx_m4.47

Finished xx_m4.47 successfully (5/100)
4.5348785081285823e+24
mpirun -np 8 py87h -p xx_m4.34 > xx_m4.34.out.txt
Fri Mar  8 17:50:23 2024    101.7 COMPLETE             xx_m4